In [54]:
#installing extra libraries on prod
!pip install zeno_etl_libs_v3==1.0.1
!pip install google-cloud-storage==1.43.0
!pip install openpyxl==3.0.9
!pip install nltk==3.6.7
!pip install apiclient==1.0.4

In [55]:
"""
Analysing Playstore reviews on daily basis
Author : neha.karekar@zeno.health
"""

'\nAnalysing Playstore reviews on daily basis\nAuthor : neha.karekar@zeno.health\n'

In [56]:
import argparse
import sys
import re
import os
import pandas as pd
import dateutil
import datetime
from dateutil.tz import gettz
import numpy as np
import time
import datetime
import os
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [57]:
sys.path.append('../../../..')

In [58]:
from zeno_etl_libs.logger import get_logger
from zeno_etl_libs.db.db import DB
from zeno_etl_libs.helper.aws.s3 import S3
from zeno_etl_libs.helper import helper

## Pass Params

In [99]:
env = "dev"
full_run = 0
email_to ="neha.karekar@zeno.health,kuldeep.singh@zeno.health"

In [100]:
os.environ['env'] = env
logger = get_logger()
logger.info(f"full_run: {full_run}")

2022-07-21 12:11:12,776 - root - INFO - full_run: 0
2022-07-21 12:11:12,776 - root - INFO - full_run: 0
2022-07-21 12:11:12,776 - root - INFO - full_run: 0


In [61]:
rs_db = DB()
rs_db_write = DB(read_only=False)
rs_db.open_connection()
rs_db_write.open_connection()
s3 = S3()

In [62]:
schema = 'prod2-generico'
table_name = 'playstore-reviews-analysis'
table_info = helper.get_table_info(db=rs_db_write, table_name=table_name, schema=schema)

In [63]:
# max of data
playstore_q = """
select
            max("review-created-at") max_exp
        from
            "prod2-generico"."playstore-reviews-analysis" 
        """
max_exp_date = rs_db.get_df(playstore_q)
max_exp_date['max_exp'].fillna(np.nan, inplace=True)
print(max_exp_date.info())
max_exp_date = max_exp_date['max_exp'].to_string(index=False)
print(max_exp_date)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   max_exp  1 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 136.0 bytes
None
2022-07-20 06:43:28


In [64]:
# params
if full_run or max_exp_date == 'NaN':
    start = '2021-01-01'
else:
    start = max_exp_date
start = dateutil.parser.parse(start)
print(start)

2021-01-01 00:00:00


In [65]:
#fetching all reviews from playstore-review table
q = f"""
    select
        "review-id",
        "review",
        "review-created-at"
    from
        "prod2-generico"."playstore-reviews"
    where
        date("review-created-at")> '{start}' 
        and review !='' and review is not null
       """
df = rs_db.get_df(q)
df['review-created-at'] = pd.to_datetime(df['review-created-at'])

In [66]:
df.head()

,review-id,review,review-created-at
0,f02d404f-911a-4787-aa0e-5034ed3782dc,THIRDCLASS SERVICE DON'T ORDER MEDICINE THIS APP,2022-07-18 05:38:55
1,d932d2c2-7a29-4cfe-be18-377703731a51,Not useful as thought,2022-07-15 17:38:25
2,0ff81d92-2aae-4415-8ef7-247642144229,"फालतू सर्विस, फालतू मे इस पर टाईम वेस्ट मत करो...",2022-07-14 16:00:25
3,704496c2-86a7-47d0-b058-4e83ac573614,good work,2022-07-09 16:00:14
4,773545a2-feaa-4f83-894f-ecbcade5869e,best service 👍,2022-07-07 07:36:50


## Define all functions

In [67]:
# finding sentiment intensity aanalyzer

def sentiment_analyser(lst):
    
    sid = SentimentIntensityAnalyzer()
    sentiment = [sid.polarity_scores(x) for x in lst]
    neg = [sid.polarity_scores(x)['neg'] for x in lst]
    neu = [sid.polarity_scores(x)['neu'] for x in lst]
    pos = [sid.polarity_scores(x)['pos'] for x in lst]
    comp = [sid.polarity_scores(x)['compound'] for x in lst]
    
    return neg[0], neu[0], pos[0], comp[0]

In [68]:
# removing stopwords
def remove_sw(sent,corpus):

    stop_words = set(stopwords.words(corpus))

    word_tokens = word_tokenize(sent)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    filtered_sentence=' '.join(filtered_sentence)
    return [filtered_sentence]

In [69]:
#Remove URL
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [70]:
def generate_N_grams(text,ngram):
    words=[word for word in text if word not in set(stopwords.words('english'))]  
    #print("Sentence after removing stopwords:",words)
    temp=zip(*[words[i:] for i in range(0,ngram)])
    ans=[' '.join(ngram) for ngram in temp]
    return ans

In [71]:
# match two list and find intersection
def matchlist(a,b):
    matches=[]
    for item_a in a:
        for item_b in b:
            if item_a == item_b:
                matches.append(item_a)
    return list(set(matches))

In [72]:
# to convert list to string
def listToString(s):
   
    # initialize an empty string
    str1 = ","
   
    # return string
    s = [str(x) for x in s] 
    return (str1.join(s))

In [73]:
#sort string
def sortstring(my_str):
        words = my_str.split(" ")  
        # sort the list  
        words.sort()  
        # display the sorted words
        newSentence = " ".join(words)
        return newSentence

In [74]:
print("Removing url")
df['content_url'] =df['review'].apply(lambda x :remove_url(x.lower()),1) 

Removing url


In [75]:
print("Removing stopwords")
sw = 'english'
df['content_sw'] =df['content_url'].apply(lambda x :remove_sw(x,sw),1) 

Removing stopwords


In [76]:
print("sentiment_analyser")
df['neg','neu','pos','comp'] =df['content_sw'].apply(lambda x :sentiment_analyser(x),1) 

sentiment_analyser


In [77]:
df[['neg','neu','pos','comp']]=   df['neg','neu','pos','comp'].apply(pd.Series)
df.drop(columns=('neg', 'neu', 'pos', 'comp'),inplace=True)

In [78]:
df['positive-ratio'] = round((df['pos']/ (df['pos']+ df ['neg'])) , 2)
df['negative-ratio'] =  round((df['neg']/ (df['pos']+ df ['neg'])) , 2)

In [79]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [80]:
df['content_lem'] = df['content_url'].apply(lambda x :lemmatize_text(x.lower()),1) 

In [81]:
df['content_2gram'] = df['content_lem'].apply(lambda x :generate_N_grams(x,2),1) 

In [82]:
df['content_1gram'] = df['content_lem'].apply(lambda x :generate_N_grams(x,1),1) 

In [83]:
#place holder
positiveValues2=defaultdict(int)
negativeValues2=defaultdict(int)
positiveValues1=defaultdict(int)
negativeValues1=defaultdict(int)

In [84]:
#get the count of every word in both the columns dataframes where sentiment="positive"
for text in df[df['positive-ratio']>df['negative-ratio']].content_lem:
    for word in generate_N_grams(text,2):
        positiveValues2[word.lower()]+=1

for text in df[df['positive-ratio']>df['negative-ratio']].content_lem:
    for word in generate_N_grams(text,1):
        positiveValues1[word.lower()]+=1

In [85]:
#get the count of every word in both the columns dataframes where sentiment="negative"
for text in df[df['positive-ratio']<=df['negative-ratio']].content_lem:
    for word in generate_N_grams(text,2):
        negativeValues2[word]+=1

#get the count of every word in both the columns of df_train and df_test dataframes where sentiment="negative"
for text in df[df['positive-ratio']<=df['negative-ratio']].content_lem:
    for word in generate_N_grams(text,1):
        negativeValues1[word]+=1

In [86]:
#focus on more frequently occuring words for every sentiment=>
#sort in DO wrt 2nd column in each of positiveValues,negativeValues
df_positive2=pd.DataFrame(sorted(positiveValues2.items(),key=lambda x:x[1],reverse=True))
df_negative2=pd.DataFrame(sorted(negativeValues2.items(),key=lambda x:x[1],reverse=True))
df_positive1=pd.DataFrame(sorted(positiveValues1.items(),key=lambda x:x[1],reverse=True))
df_negative1=pd.DataFrame(sorted(negativeValues1.items(),key=lambda x:x[1],reverse=True))

In [87]:
#convert grams to list
positive_grams2=df_positive2[0][:20].tolist()
positive_grams1=df_positive1[0][:20].tolist()
negative_grams2=df_negative2[0][:20].tolist()
negative_grams1=df_negative1[0][:20].tolist()

In [88]:
df['top_grams2']= np.where(df['positive-ratio']>df['negative-ratio'],df['content_2gram'].apply(lambda x :matchlist(x,positive_grams2),1) ,df['content_2gram'].apply(lambda x :matchlist(x,negative_grams2),1) )

In [89]:
df['top_grams1']= np.where(df['positive-ratio']>df['negative-ratio'],df['content_1gram'].apply(lambda x :matchlist(x,positive_grams1),1) ,df['content_1gram'].apply(lambda x :matchlist(x,negative_grams1),1) )

In [90]:
df['top-grams-one']= df['top_grams1'].apply(lambda x :listToString(x))

In [91]:
df['top-grams-two']= df['top_grams2'].apply(lambda x :listToString(x))
df['top-grams-two']= df['top-grams-two'].apply(lambda x :sortstring(x),1)

In [92]:
df.head()

,review-id,review,review-created-at,content_url,content_sw,neg,neu,pos,comp,positive-ratio,negative-ratio,content_lem,content_2gram,content_1gram,top_grams2,top_grams1,top-grams-one,top-grams-two
0,f02d404f-911a-4787-aa0e-5034ed3782dc,THIRDCLASS SERVICE DON'T ORDER MEDICINE THIS APP,2022-07-18 05:38:55,thirdclass service dont order medicine this app,[thirdclass service dont order medicine app],0.0,1.000,0.000,0.0000,NaN,NaN,"[thirdclass, service, dont, order, medicine, t...","[thirdclass service, service dont, dont order,...","[thirdclass, service, dont, order, medicine, app]",[order medicine],"[app, service, order, medicine, dont]","app,service,order,medicine,dont",medicine order
1,d932d2c2-7a29-4cfe-be18-377703731a51,Not useful as thought,2022-07-15 17:38:25,not useful as thought,[useful thought],0.0,0.256,0.744,0.4404,1.0,0.0,"[not, useful, a, thought]",[useful thought],"[useful, thought]",[],[],,
2,0ff81d92-2aae-4415-8ef7-247642144229,"फालतू सर्विस, फालतू मे इस पर टाईम वेस्ट मत करो...",2022-07-14 16:00:25,medicine,[medicine],0.0,1.000,0.000,0.0000,NaN,NaN,[medicine],[],[medicine],[],[medicine],medicine,
3,704496c2-86a7-47d0-b058-4e83ac573614,good work,2022-07-09 16:00:14,good work,[good work],0.0,0.256,0.744,0.4404,1.0,0.0,"[good, work]",[good work],"[good, work]",[],[good],good,
4,773545a2-feaa-4f83-894f-ecbcade5869e,best service 👍,2022-07-07 07:36:50,best service,[best service],0.0,0.192,0.808,0.6369,1.0,0.0,"[best, service]",[best service],"[best, service]",[best service],"[service, best]","service,best",best service


In [93]:
final_ana = df[['review-id','review-created-at','positive-ratio','negative-ratio','top-grams-one','top-grams-two']]

In [94]:
#etl columns
final_ana['created-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
final_ana['updated-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
final_ana['created-by'] = 'etl-automation'
final_ana['updated-by'] = 'etl-automation'

c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying

In [95]:
if isinstance(table_info, type(None)):
    raise Exception(f"table: {table_name} do not exist, create the table first")
else:
    logger.info(f"Table:{table_name} exists")

2022-07-21 11:54:44,994 - root - INFO - Table:playstore-reviews-analysis exists
2022-07-21 11:54:44,994 - root - INFO - Table:playstore-reviews-analysis exists


In [96]:
truncate_query = f''' DELETE FROM "{schema}"."{table_name}" where "review-created-at" >'{start}' '''
print(truncate_query)
rs_db_write.execute(truncate_query)

 DELETE FROM "prod2-generico"."playstore-reviews-analysis" where "review-created-at" >'2021-01-01 00:00:00' 


In [97]:
s3.write_df_to_db(df=final_ana[table_info['column_name']], table_name=table_name, db=rs_db_write,
                      schema=schema)

S3 object(uri: s3://aws-glue-temporary-921939243643-ap-south-1/temp_1658384686210.csv) delete response: {'ResponseMetadata': {'RequestId': 'F1DXWCFQCDEA5T9G', 'HostId': 'aJWnhbsTHn84/63Qnnj31AKmwfshcZIryC1fmLYKH3k4VGpA6zW19KQZulYO3LsmYcVS1ayoLU0=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'aJWnhbsTHn84/63Qnnj31AKmwfshcZIryC1fmLYKH3k4VGpA6zW19KQZulYO3LsmYcVS1ayoLU0=', 'x-amz-request-id': 'F1DXWCFQCDEA5T9G', 'date': 'Thu, 21 Jul 2022 06:24:48 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}
S3 object(uri: s3://aws-glue-temporary-921939243643-ap-south-1/temp_1658384686210.csv) delete response: {'ResponseMetadata': {'RequestId': 'YAZAD7XD4HRHKH3H', 'HostId': '/CbgzbIeZliqs0fidBWxokTYQZlOJ5OPtqQ3Gd7FgjY8rpuq5h9aXr6zoTHw+P4ORVL7lHpe+bU=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': '/CbgzbIeZliqs0fidBWxokTYQZlOJ5OPtqQ3Gd7FgjY8rpuq5h9aXr6zoTHw+P4ORVL7lHpe+bU=', 'x-amz-request-id': 'YAZAD7XD4HRHKH3H', 'date': 'Thu, 21 Jul 2022 06:24:49 GMT', 'server': 'AmazonS3'}, 'RetryA

In [98]:
# Closing the DB Connection
rs_db.close_connection()
rs_db_write.close_connection()


Redshift DB connection closed successfully.
Redshift DB connection closed successfully.
